# Packages Loading

In [1]:
from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import os

import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Spark Config Setting

In [2]:
SPARK_JAR_DIR = os.getenv("SPARK_JAR_DIR", "/opt/spark/jars")

SPARK_STAND_ALONE_CORES = {
    "spark.cores.max": 3
}

SPARK_DRIVER_CONFIGS = {
    "spark.driver.memory": "1g"
}

SPARK_EXECUTOR_CONFIGS = {
    "spark.executor.cores": 1,
    "spark.executor.instances": 1,
    "spark.executor.memory": "1g",
}

spark = SparkSession.builder \
    .appName("etl_snapshot") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", f"{SPARK_JAR_DIR}/mysql-connector-java-8.0.28.jar") \
    .config("spark.ui.port", "4040") \
    .config(map=SPARK_DRIVER_CONFIGS) \
    .config(map=SPARK_EXECUTOR_CONFIGS) \
    .config(map=SPARK_STAND_ALONE_CORES) \
    .getOrCreate()

spark.sparkContext.setJobGroup("snap_monthly_user_purchase_frequency_and_monetary", "The overall quartiles of the total purchase amount and total number of purchases per user over one year", interruptOnCancel=False)

# RDB Connection Info

In [3]:
DB_CONN_INFO = {
    "user": "root",
    "password": "root",
    "url": "jdbc:mysql://mysql:3306/mysql",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Data Sourcing Query

### 1. 검색 조건 일자의 전체 데이터 개수 확인

In [4]:
start_de = "2016-06-01"
end_de = "2022-07-31"

### 2. 검색 조건에 부합하는 전체 데이터 셋 집합 카운트 수 산출

In [5]:
count_sql = f"""
    SELECT COUNT(*)
    FROM dm.dm_f_ordr
    WHERE ORDR_DE BETWEEN '{start_de}' AND '{end_de}'
"""

# 검색할 범주의 전체 행의 개수 산출 (파티션 수를 균등하게 분할하여 읽기 위한 전체 개수 탐색)
row_count_sql = spark.read.format("jdbc") \
    .options(**DB_CONN_INFO) \
    .option("query", count_sql) \
    .load()

row_count = row_count_sql.first()[0]

In [6]:
row_count_sql.explain(extended="formatted")

== Physical Plan ==
* Scan JDBCRelation((
    SELECT COUNT(*)
    FROM dm.dm_f_ordr
    WHERE ORDR_DE BETWEEN '2016-06-01' AND '2022-07-31'
) SPARK_GEN_SUBQ_0) [numPartitions=1]  (1)


(1) Scan JDBCRelation((
    SELECT COUNT(*)
    FROM dm.dm_f_ordr
    WHERE ORDR_DE BETWEEN '2016-06-01' AND '2022-07-31'
) SPARK_GEN_SUBQ_0) [numPartitions=1]  [codegen id : 1]
Output [1]: [COUNT(*)#0L]
ReadSchema: struct<COUNT(*):bigint>




### 3. 여러 개의 파티션으로 균등 분할하여 데이터 Read

In [7]:
dbtable = f"""(
    SELECT CUST_ID
         , ORDR_DE
         , GDS_ORDR_GRAMT
         , ROW_NUMBER () OVER () AS row_num
      FROM dm.dm_f_ordr
     WHERE ORDR_DE BETWEEN '{start_de}' AND '{end_de}'
) as subquery"""

sdf = spark.read.format("jdbc") \
    .options(**DB_CONN_INFO) \
    .option("numPartitions", 6) \
    .option("dbtable", dbtable) \
    .option("partitionColumn", "row_num") \
    .option("lowerBound", 1) \
    .option("upperBound", row_count) \
    .load()

sdf.cache()

DataFrame[CUST_ID: int, ORDR_DE: date, GDS_ORDR_GRAMT: bigint, row_num: decimal(20,0)]

In [8]:
sdf.show()

+-------+----------+--------------+-------+
|CUST_ID|   ORDR_DE|GDS_ORDR_GRAMT|row_num|
+-------+----------+--------------+-------+
|  31256|2021-06-05|         64894|      1|
|  26434|2020-11-09|        399291|      2|
|  17423|2019-02-28|        390872|      3|
|  43395|2020-10-24|        131010|      4|
|  62403|2022-04-21|        263498|      5|
|  45912|2018-02-08|        308287|      6|
|  49334|2018-10-07|        261020|      7|
|   5717|2020-07-18|        372006|      8|
|  75454|2018-12-23|        364607|      9|
|  44294|2019-01-26|        266197|     10|
|  36742|2021-04-23|        156340|     11|
|  50914|2018-09-23|        103125|     12|
|  40261|2021-11-16|        217838|     13|
|  27813|2021-10-04|        561276|     14|
|  97451|2022-03-06|        249606|     15|
|  18974|2020-11-19|        301996|     16|
|  26522|2021-10-04|       1091334|     17|
|  59659|2021-04-23|        239181|     18|
|  29140|2020-12-09|        398409|     19|
|  66591|2018-02-22|        1503

In [9]:
# 파티션별 (파티션 인덱스, 데이터) 개수 => 6개 파티션으로 데이터를 분할해서 읽음 (각 파티션에는 customer_id별로 모아져있음)
sdf.rdd.mapPartitionsWithIndex(lambda idx, it: [(idx, sum(1 for _ in it))]).collect()

[(0, 135995), (1, 135993), (2, 135993), (3, 135993), (4, 135993), (5, 135997)]

In [10]:
# 파티션 개수
sdf.rdd.getNumPartitions()

6

In [11]:
# schema 출력
sdf.printSchema()

root
 |-- CUST_ID: integer (nullable = true)
 |-- ORDR_DE: date (nullable = true)
 |-- GDS_ORDR_GRAMT: long (nullable = true)
 |-- row_num: decimal(20,0) (nullable = true)



# 매월 스냅샷 쿼리 (멱등성 처리)

In [12]:
date_range = []

for dt in pd.date_range(start=start_de, end=end_de, freq='MS'):
    snapshot_stdr_de = dt.date() 
    one_year_ago_de = snapshot_stdr_de - relativedelta(years=1) # 1년 전의 날짜 구하기
    one_day_before_de = snapshot_stdr_de - relativedelta(days=1) # 하루 전의 날짜 구하기

    date_range.append({
        "snapshot_stdr_de": snapshot_stdr_de,
        "one_year_ago_de": one_year_ago_de,
        "one_day_before_de": one_day_before_de
    })

In [13]:
# 빈 Pandas DataFrame 생성
result_pdf = pd.DataFrame(
    columns=[
        "ORDR_DE", "INTERVAL_START_DE", "INTERVAL_END_DE",
        "ALL_USER_GDS_ORDR_GRAMT_Q1", "ALL_USER_GDS_ORDR_GRAMT_Q2", "ALL_USER_GDS_ORDR_GRAMT_Q3",
        "ALL_USER_PURCHASE_CNT_Q1", "ALL_USER_PURCHASE_CNT_Q2", "ALL_USER_PURCHASE_CNT_Q3"
    ]
)

for row in date_range:

    sdf_2 = sdf.filter(F.col("ORDR_DE").between(row["one_year_ago_de"], row["one_day_before_de"])).select(["CUST_ID", "ORDR_DE", "GDS_ORDR_GRAMT"])
    
    sdf_3 = sdf_2.groupBy("CUST_ID").agg(
        F.sum("GDS_ORDR_GRAMT").alias("USER_GDS_ORDR_GRAMT"),
        F.count("*").alias("USER_PURCHASE_CNT")
    )

    user_gds_ordr_gramt = sdf_3.approxQuantile(col="USER_GDS_ORDR_GRAMT", probabilities=[0.25, 0.5, 0.75], relativeError=0)
    if not user_gds_ordr_gramt:
        user_gds_ordr_gramt = [pd.NA, pd.NA, pd.NA]
        
    user_purchase_cnt = sdf_3.approxQuantile(col="USER_PURCHASE_CNT", probabilities=[0.25, 0.5, 0.75], relativeError=0)
    if not user_purchase_cnt:
        user_purchase_cnt = [pd.NA, pd.NA, pd.NA]
    
    data = [{
        'ORDR_DE': row["snapshot_stdr_de"],
        "INTERVAL_START_DE": row["one_year_ago_de"],
        "INTERVAL_END_DE": row["one_day_before_de"],
        "ALL_USER_GDS_ORDR_GRAMT_Q1": user_gds_ordr_gramt[0],
        "ALL_USER_GDS_ORDR_GRAMT_Q2": user_gds_ordr_gramt[1],
        "ALL_USER_GDS_ORDR_GRAMT_Q3": user_gds_ordr_gramt[2],
        "ALL_USER_PURCHASE_CNT_Q1": user_purchase_cnt[0],
        "ALL_USER_PURCHASE_CNT_Q2": user_purchase_cnt[1],
        "ALL_USER_PURCHASE_CNT_Q3": user_purchase_cnt[2]
    }]
    
    result_pdf = pd.concat(objs=[result_pdf, pd.DataFrame(data)], ignore_index=True)

In [14]:
sdf_2.show()

+-------+----------+--------------+
|CUST_ID|   ORDR_DE|GDS_ORDR_GRAMT|
+-------+----------+--------------+
|  62403|2022-04-21|        263498|
|  40261|2021-11-16|        217838|
|  27813|2021-10-04|        561276|
|  97451|2022-03-06|        249606|
|  26522|2021-10-04|       1091334|
|  89268|2022-06-14|        484149|
|   9142|2021-08-11|        894014|
|  35635|2021-08-17|        159304|
|  10354|2022-03-28|        320944|
|   9098|2021-07-10|        244603|
|   4947|2022-06-08|       2304014|
|  24360|2021-09-04|        212495|
|  85994|2022-03-15|        107074|
|  73911|2021-08-12|        631122|
|   9333|2021-09-14|        430180|
|  81864|2021-07-12|        655805|
|  55615|2021-10-08|        161544|
|  46242|2021-12-27|        297547|
|  87476|2022-06-08|        911511|
|  36973|2021-12-27|        259551|
+-------+----------+--------------+
only showing top 20 rows



In [15]:
sdf_3.describe().show()

+-------+------------------+-------------------+------------------+
|summary|           CUST_ID|USER_GDS_ORDR_GRAMT| USER_PURCHASE_CNT|
+-------+------------------+-------------------+------------------+
|  count|             37775|              37775|             37775|
|   mean| 49834.32158835208| 4292998.7461283915| 7.924394440767704|
| stddev|28971.336529102467|  6122497.456485517|10.469767621987682|
|    min|                 3|              29257|                 1|
|    max|             99998|          108079558|               172|
+-------+------------------+-------------------+------------------+



In [16]:
# 파티션별 (파티션 인덱스, 데이터) 개수 => groupBy흘 실행했기 때문에 하나의 노드, 하나의 파티션으로 통합
sdf_3.rdd.mapPartitionsWithIndex(lambda idx, it: [(idx, sum(1 for _ in it))]).collect()

[(0, 18652), (1, 19123)]

In [17]:
int_type_cols = ("ALL_USER_GDS_ORDR_GRAMT_Q1", "ALL_USER_GDS_ORDR_GRAMT_Q2", "ALL_USER_GDS_ORDR_GRAMT_Q3", "ALL_USER_PURCHASE_CNT_Q1", "ALL_USER_PURCHASE_CNT_Q2", "ALL_USER_PURCHASE_CNT_Q3")

for col in int_type_cols:
    result_pdf[col] = result_pdf[col].astype("Int64")

In [18]:
result_pdf.to_excel("snap_monthly_user_purchase_frequency_and_monetary.xlsx", index=False)

In [19]:
result_pdf.tail(10)

,ORDR_DE,INTERVAL_START_DE,INTERVAL_END_DE,ALL_USER_GDS_ORDR_GRAMT_Q1,ALL_USER_GDS_ORDR_GRAMT_Q2,ALL_USER_GDS_ORDR_GRAMT_Q3,ALL_USER_PURCHASE_CNT_Q1,ALL_USER_PURCHASE_CNT_Q2,ALL_USER_PURCHASE_CNT_Q3
64,2021-10-01,2020-10-01,2021-09-30,443391,1406268,4200380,1,3,8
65,2021-11-01,2020-11-01,2021-10-31,458550,1431666,4308453,1,3,8
66,2021-12-01,2020-12-01,2021-11-30,480030,1477286,4406727,1,3,8
67,2022-01-01,2021-01-01,2021-12-31,512593,1564760,4559700,1,3,8
68,2022-02-01,2021-02-01,2022-01-31,530397,1611683,4622174,2,3,8
69,2022-03-01,2021-03-01,2022-02-28,547219,1658098,4714276,2,3,8
70,2022-04-01,2021-04-01,2022-03-31,579798,1728458,4872096,2,3,9
71,2022-05-01,2021-05-01,2022-04-30,596704,1803707,4996775,2,4,9
72,2022-06-01,2021-06-01,2022-05-31,617182,1919287,5222918,2,4,9
73,2022-07-01,2021-07-01,2022-06-30,612087,1961551,5418590,2,4,10


# Stop SparkSession

In [20]:
spark.stop()